# Control Systems 1, NB07: Time-Domain specifications and PID

© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers various transfer function formulations, and how they can be used to graphically determine the magnitude and phase of the transfer function. Furthermore, the effects of poles and zeros on the output response are briefly covered.

Authors:
- Jonas Holinger; jholinger@ethz.ch
- Shubham Gupta; shugupta@ethz.ch

## Learning Objectives

After completing this notebook, you should be able to:

1. Recognize when the steady state error occurs and how eliminate it
2. Understand different time-domain specifications for first and second order system
3.Understand dominant pole approximations  


In [2]:
%pip install numpy matplotlib scipy ipywidgets control IPython sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.7 MB/s eta 0:00:00


In [4]:
import control as ct
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from scipy.integrate import odeint
from IPython.display import display, clear_output, Math
from ipywidgets import interactive

# Steady State Error


The steady state error is the difference between the system refrence signal $r(t)$ and the closed-loop output $y(t)$ for $ \displaystyle \lim_{t \to \infty }$.<br>
From the final value theorem, we know that we can calculate the steady state error with: $$ e_{ss}=  \displaystyle \lim_{t \to \infty } e(t) =  \displaystyle \lim_{s \to 0} s ⋅ E(s)$$
<br>
### Example
We have an open-loop transfer function $C(s)P(s)$ with an integrator, there is a pole at $s=0$. Let’s define the open-loop transfer function $C(s)P(s)$ as $$L(s)= \frac{K_{Bode}}{s(s+2)}$$
K is our systems's bode-gain, the s term is our systems integrator which makees our system a **Type 1** system and (s+2) as an additional stable pole.
The input to the system is a first order ramp ($q=1$) r(t) = t. The corresponding Input in frequency domain is: $$R(s) = \frac{1}{s^2}$$
To calulcate the error signal $E(s)$ we use the sensitivity function, which is known from previous lectures. The sensitvity funciton maps the refrence input $r$ to the error $e$ $$ S(s) = \frac{1}{1+P(s)C(s)}=\frac{1}{1+L(s)}$$.

We can multiply the sensitivity funciton $S$ with our input $R(s)$ and apply the final value theorem to get our steady state error
$$e_{ss}=\displaystyle \lim_{s \to 0} s ⋅ E(s)=\displaystyle \lim_{s \to 0} s ⋅ R(s)⋅\frac{1}{1+L(S)} = \displaystyle \lim_{s \to 0} s ⋅ \frac{\frac{1}{s^2}}{1+\frac{K}{s(s+2)}} $$

The steady state error we get after applying the limit is  $e_{ss}=\frac{2}{K}$ which corresponds to inverse of our bode-gain $K_{Bode}$. If the system is stable and q=0, the Bode gain corresponds to the steady state gain to an unit step input and is called DC gain.

If we know add a second integrator to our system $C(s)=s^2$ and calcualte our steady-state error again $$e_{ss}= \displaystyle \lim_{s \to 0} s ⋅ \frac{\frac{1}{s^2}}{1+\frac{K}{s^2(s+2)}}$$
we see that our steady-state error is exactly zero.<br><br> For conclusion, to get a steady state error of zero, you need to have an integrator more than the order of your ramp. To get a small steady state error you can increase the Bode gain.  To avoid calculating the steady state error each time, you can use the following table:



Bode gain and type of system

| $ e_{ss} $ | $ q = 0 $                   | $ q = 1 $            | $ q = 2 $            |
|-------------|-----------------------------|----------------------|----------------------|
| Type 0      | $ \frac{1}{1 + K_{\text{Bode}}} $ | $ \infty $           | $ \infty $           |
| Type 1      | $ 0 $                        | $ \frac{1}{K_{\text{Bode}}} $ | $ \infty $           |
| Type 2      | $ 0 $                        | $ 0 $                | $ \frac{1}{K_{\text{Bode}}} $ |

### Visualiation
Try different combinations of system type and input ramp and observe wether you have a steady state error. Try changing the bode gain and see how the systems react to you changes.

In [6]:
def simulate_system(system_type, input_type, bode_gain):
    # Time vector
    t = np.linspace(0, 125, 1000)  # Adjusted for a clearer response over a shorter time

    # Define the stable plant P(s) = 1 / (s + 10)
    P = ct.TransferFunction([1], [1, 10])

    # Define the controller C(s) based on system type
    if system_type == "Type 0":
        # Proportional controller with capped gain for stability
        C = ct.TransferFunction([min(bode_gain, 5)], [1])  # Gain limited to ensure stability
    elif system_type == "Type 1":
        # Proportional-Integral (PI) controller K(s + 1) / s
        C = ct.TransferFunction([min(bode_gain, 1), min(bode_gain, 1)], [1, 0])  # PI controller
    elif system_type == "Type 2":
        # Proportional-Derivative-Integral (PID) controller for enhanced stability
        C = ct.TransferFunction([min(bode_gain, 0.003), 0.5 * bode_gain, min(bode_gain, 0.1)], [1, 0, 0])


    # Create the open-loop system by multiplying C and P
    open_loop_system = C * P

    # Calculate the closed-loop transfer function with unity feedback
    closed_loop_system = ct.feedback(open_loop_system, 1)  # Unity feedback configuration

    # Define the input based on the input type
    if input_type == "Unit Step":
        input_signal = np.ones_like(t)  # Step input
    elif input_type == "Unit Ramp":
        input_signal = t  # Ramp input
    elif input_type == "Unit Parabola":
        input_signal = 0.5 * t**2  # Parabolic input

    # Simulate system response to the input
    t_out, y_out = ct.forced_response(closed_loop_system, T=t, U=input_signal)

    # Plotting input and output with dynamic y-axis limits
    plt.figure(figsize=(12, 6))
    plt.plot(t, input_signal, 'r--', label='Input Signal', linewidth=2)
    plt.plot(t_out, y_out, 'b-', label='Output Response (Closed Loop)', linewidth=2)



    plt.title(f'Input and Output of {system_type} System to {input_type} (Closed Loop)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Displaying closed-loop transfer function
    print("Closed-Loop Transfer Function T(s):")
    print(closed_loop_system)

# Create dropdowns for system type and input type
system_type_dropdown = widgets.Dropdown(
    options=["Type 0", "Type 1", "Type 2"],
    value="Type 0",
    description='System Type:',
)

input_type_dropdown = widgets.Dropdown(
    options=["Unit Step", "Unit Ramp", "Unit Parabola"],
    value="Unit Step",
    description='Input Type:',
)

# Create text input for bode gain
bode_gain_input = widgets.FloatText(
    value=1.0,
    description='Bode Gain K:',
    step=0.1
)

# Interactive output
interactive_plot = interactive(simulate_system,
                               system_type=system_type_dropdown,
                               input_type=input_type_dropdown,
                               bode_gain=bode_gain_input)
display(interactive_plot)


interactive(children=(Dropdown(description='System Type:', options=('Type 0', 'Type 1', 'Type 2'), value='Type…

### Example of Steady State error

# Time Domain Speicifications

##First-Order system

In both the time domain and the frequency domain, we can analyze a system's response. In the time domain, we typically separate the response into two stages: the transient stage and the steady-state stage. Now we want to examine different behaviors in our time domain.

Let’s consider a stable first-order system: <br>$$G(s)=\frac{K}{τs+1}$$<br>
For an inital condition of $x(0)=0$, the solution is given by: $$y(t)=K-e^{\frac{-t}{\tau}}$$.

We are interested in the following time-domain specifications for a stable first-order system:


**DC gain**
<br>
The Steady-state or DC gain **$y_{ss}$** is a measure of how much the systems output responds to an input after the transients effects have died out.
In our first order system the DC gain corresponds to K.

**Time constant**
<br>
The time constant **$\tau$** measures the speed of the system's response.
 It is defined as the time required for the system's unit step response to reach approximately 63% of its final value.

**Settling time**
<br>
The settling time **$T_d$** is the time it takes for the output to remain within $d\%$ of the final value. The settling time time can be approximated using the time constant $τ$ using the formula $$T_d=τ \cdot log(100/d)$$ For $5\%$ the approximation is $T_d=3\tau$



In [ ]:
def plot_first_order_system(K=3, tau=2):
    # Create the transfer function
    sys = ct.TransferFunction([K], [tau, 1])

    # Time vector for simulation
    t = np.linspace(0, 10 * tau, 1000)

    # Step response
    t, y = ct.step_response(sys, T=t)

    tau_time = tau
    yss = K  # Steady-state value (DC Gain)
    settling_time = -np.log(0.05) * tau  # Settling time for 5%

    y_tau = K * (1 - np.exp(-tau_time / tau))
    y_settling = K * (1 - np.exp(-settling_time / tau))

    # Plotting
    plt.figure(figsize=(10, 6))

    # Plot step response
    plt.plot(t, y, label="Step Response", color="blue")

    # Plot step input
    plt.plot(t, np.ones_like(t), label="Step Input", color="orange", linestyle='--')

    plt.axhline(yss, color="red", linestyle="--")

    # Draw vertical lines for τ and settling time
    plt.plot([tau_time, tau_time], [0, y_tau], color="green", linestyle="--")
    plt.plot([settling_time, settling_time], [0, y_settling], color="purple", linestyle="--")

    # Add labels for time specifications directly under the lines
    plt.text(tau_time, 0, f"τ    ", color="green", ha="center", va="bottom", fontsize=10, fontweight="bold")
    plt.text(settling_time, 0, f"Settling Time    ", color="purple", ha="center", va="bottom", fontsize=10, fontweight="bold")

    # Add label for DC gain just above the line
    plt.text(0, yss, f"DC Gain", color="red", ha="left", va="bottom", fontsize=10, fontweight="bold")

    # Add labels and legend
    plt.xlabel("Time (s)")
    plt.ylabel("Response")
    plt.title(f"Step Response of First-Order System: G(s) = {K}/({tau}s + 1)")
    plt.legend(loc="best")
    plt.grid(True)
    plt.ylim(0, 1.2 * yss)
    plt.show()

# Create interactive widgets for K and tau
K_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description="K:")
tau_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description="τ:")

# Display the interactive plot
interactive_plot = interactive(plot_first_order_system, K=K_slider, tau=tau_slider)
display(interactive_plot)


interactive(children=(FloatSlider(value=1.0, description='K:', max=10.0, min=0.1), FloatSlider(value=1.0, desc…

## Second-Order system

For a second order system in the form:
$$G(s) = \frac{\omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2}
\quad \Longleftrightarrow \quad
\begin{cases}
    \dot{x} = \begin{bmatrix} 0 & 1 \\ -\omega_n^2 & -2 \zeta \omega_n \end{bmatrix} x + \begin{bmatrix} 0 \\ \omega_n^2 \end{bmatrix} u \\
    y = x
\end{cases}
$$
Imposing our zero inital condition and solving the algebra we get as our solution for an underamped system  :
$$y(t) = 1 - \frac{1}{\cos \varphi} e^{\sigma t} \cos(\omega t + \varphi), \quad t \geq 0.
$$
with $ϕ=arctan(\frac{σ}{ω})$
<br>
<br>
$\omega_n$ denotes the natural frequency of our system, sometimes also called the eigenfrequency. The natural frequency is defined as $\omega_n^2=\sigma^2+\omega^2$ an can be calculated from the pole of the system $s=\sigma+j\omega$

 The damping factor $\zeta = \frac{\sigma}{\omega_n}$ can be used to catogorise our system into the following cases:
<br>
*   $ζ<0$, underdamped
<br>
*   $ζ=0$, critically damped
<br>
*   $ζ>0$, overdamped

We can see that the locaction of the pole in our system determines the natural frequency, the damping ratio and the time domain specifications

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interactive, FloatSlider, VBox

def plot_damping_responses(omega_n):
    damping_ratios = {
        "Underdamped (ζ < 1)": 0.5,
        "Critically Damped (ζ = 1)": 1.0,
        "Overdamped (ζ > 1)": 1.5
    }

    # Time vector for the step response
    t = np.linspace(0, 10, 500)

    plt.figure(figsize=(12, 8))

    for label, zeta in damping_ratios.items():
        # Define the transfer function
        num = [omega_n**2]
        den = [1, 2*zeta*omega_n, omega_n**2]
        system = signal.TransferFunction(num, den)

        # Compute the step response
        t, response = signal.step(system, T=t)

        # Plot the step response
        plt.plot(t, response, label=label)

    plt.title('Step Responses of Second Order Systems')
    plt.xlabel('Time [s]')
    plt.ylabel('Response')
    plt.axhline(1, color='gray', linestyle='--', linewidth=0.8)  # Final value line
    plt.grid(True)
    plt.legend()
    plt.show()

# Slider for natural frequency ωₙ
omega_n_slider = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='ωₙ')

# Interactive plot
interactive_plot = interactive(plot_damping_responses, omega_n=omega_n_slider)

# Display the interactive plot
display(interactive_plot)


interactive(children=(FloatSlider(value=1.0, description='ωₙ', max=10.0, min=0.1), Output()), _dom_classes=('w…

### Time-Domain Specifications for a second order system


**Steady-state Gain**

The Steady-state Gain remains the same with that for the first order system.
The time constant $\tau$ can be calculated using the formula:  $$τ=\frac{1}{{|\sigma|}}$$.

**Settling Time**

The settling time $$T_d= \tau\cdot log(\frac{100}{d})$$ is also the same as in the first order system.
 For an overdamped system, where the poles are real and distinct, the settling time $T_d$ should be calculated using the pole with the largest time constant $\tau$. This pole represents the slowest decay rate, which dominates the system's response.


**Time to peak**

The time to peak is the duration it takes for the system response to reach its maximum value after a unit step input. For underdamped systems, it is given by:
$$T_p=\frac{\pi}{\omega}$$

**Hint:**
To calculate $\omega$ given natural frequency $w_n$ and damping coefficent $\zeta$ use  formula: $\omega=\omega_n\sqrt{1-\zeta^2}$


**Peak Overshoot Ratio**

The peak overshoot ratio is the maximum amount, by which the response exceeds the steady-state value. It can be expressed as:<br><br> $$M_p=e^{\frac{\sigma\pi}{\omega}}$$ <br> Alternatively, it can be related to the damping ratio $\zeta$ using: $\zeta^2=\frac{(lnM_p)^2}{\pi^2+(lnM_p)^2}$

**Rise Time**

The rise time is defined as the time required for the response to rise from 0% to 100% of the steady-state value.

$T_{100\%}=\frac{\frac{\pi}{2}-\phi}{\omega}\approx \frac{\pi}{2\omega_n}$

Imposing a restriction on the time domain specification automatically imposes a restriction for the location of the pole in the complex plane

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interactive, ToggleButton, FloatSlider

# Function to calculate and display the response with time-domain specifications
def plot_step_response(omega_n, zeta, show_time_to_peak, show_peak_overshoot, show_rise_time, show_settling_time, show_time_constant):
    # Define the transfer function
    num = [omega_n**2]
    den = [1, 2*zeta*omega_n, omega_n**2]
    system = signal.TransferFunction(num, den)

    # Time vector for the step response
    t = np.linspace(0, 5, 500)
    t, response = signal.step(system, T=t)

    # Calculate time-domain specifications
    # Time to Peak (Tp)
    Tp = np.pi / (omega_n * np.sqrt(1 - zeta**2)) if zeta < 1 else np.nan

    # Peak Overshoot Ratio (%OS)
    OS = np.exp(-zeta * np.pi / np.sqrt(1 - zeta**2)) * 100 if zeta < 1 else 0

    # Rise Time (T100%)
    Tr = (1.8 / omega_n) if zeta < 0.69 else (np.nan if zeta > 1 else 0)

    # Settling Time (Ts)
    Ts = 4 / (zeta * omega_n) if zeta > 0 else np.nan

    # Time Constant (tau)
    tau = 1 / (zeta * omega_n) if zeta > 0 else np.nan

    # Plot the step response
    plt.figure(figsize=(12, 6))

    # Step Response Plot
    plt.subplot(1, 2, 1)
    plt.plot(t, response, label='Step Response')

    # Plot time-domain specifications if toggled on
    if show_time_to_peak and not np.isnan(Tp):
        plt.text(Tp, 0, f'Tp = {Tp:.2f} s', color='red', fontsize=10, ha='right')
        plt.plot([Tp, Tp], [0, response[np.where(t >= Tp)[0][0]]], color='r', linestyle='--')

    if show_peak_overshoot:
        plt.plot(Tp, response.max(), 'ro', label=f'Peak Overshoot = {OS:.2f}%')
        plt.text(Tp, response.max(), f'OS = {OS:.2f}%', color='red', fontsize=10, ha='center', va='bottom')

    if show_rise_time and not np.isnan(Tr):
        plt.text(Tr,0, f'Tr = {Tr:.2f} s', color='green', fontsize=10, ha='right')
        plt.plot([Tr, Tr], [0, response[np.where(t >= Tr)[0][0]]], color='g', linestyle='--')

    if show_settling_time and not np.isnan(Ts):
        plt.text(Ts, 0, f'Ts = {Ts:.2f} s', color='magenta', fontsize=10, ha='right')
        plt.plot([Ts, Ts], [0, response[np.where(t >= Ts)[0][0]]], color='m', linestyle='--')

    if show_time_constant and not np.isnan(tau):
        plt.text(tau, 0, f'τ = {tau:.2f} s', color='cyan', fontsize=10, ha='right')
        plt.plot([tau, tau], [0, response[np.where(t >= tau)[0][0]]], color='c', linestyle='--')

    plt.title('Step Response of Second Order System')
    plt.xlabel('Time [s]')
    plt.ylabel('Response')
    plt.legend()
    plt.grid(True)
    plt.ylim(0, 1.1 * response.max())  # Adjust y-limits for better visibility of text

    # Calculate and plot poles
    poles = np.roots(den)

    # Pole Plot
    plt.subplot(1, 2, 2)
    plt.scatter(np.real(poles), np.imag(poles), color='blue', marker='x')
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.title('Poles of the System')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Sliders for omega_n and zeta
omega_n_slider = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='ωₙ')
zeta_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description='ζ')

# Toggle buttons for displaying time-domain specifications
show_time_to_peak_btn = ToggleButton(value=False, description='Time to Peak', button_style='info')
show_peak_overshoot_btn = ToggleButton(value=False, description='Peak Overshoot', button_style='info')
show_rise_time_btn = ToggleButton(value=False, description='Rise Time (T100%)', button_style='info')
show_settling_time_btn = ToggleButton(value=False, description='Settling Time', button_style='info')
show_time_constant_btn = ToggleButton(value=False, description='Time Constant', button_style='info')

# Layout for interactive plot
interactive_plot = interactive(
    plot_step_response,
    omega_n=omega_n_slider,
    zeta=zeta_slider,
    show_time_to_peak=show_time_to_peak_btn,
    show_peak_overshoot=show_peak_overshoot_btn,
    show_rise_time=show_rise_time_btn,
    show_settling_time=show_settling_time_btn,
    show_time_constant=show_time_constant_btn
)

display(interactive_plot)


interactive(children=(FloatSlider(value=1.0, description='ωₙ', max=10.0, min=0.1), FloatSlider(value=0.5, desc…

In [ ]:
import control as ct
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive

# Function to compute pole locations based on constraints
def plot_poles(K, zeta, T_s, PO, T_r):
    # Clear previous plots
    plt.figure(figsize=(10, 10))

    # Generate a grid of pole locations
    real_parts = np.linspace(-3, 0, 100)
    imag_parts = np.linspace(-3, 3, 100)
    R, I = np.meshgrid(real_parts, imag_parts)

    # Calculate the allowed regions based on settling time
    # Settling time condition: T_s = 4 / (zeta * omega_n)
    omega_n_settle = 4 / T_s

    # Peak Overshoot condition with validity check
    if PO > 0 and PO < 100:
        # Calculate zeta from PO
        zeta_from_PO = -np.log(PO / 100) * np.sqrt(1 - (1 / (PO / 100) ** 2)) / np.pi
    else:
        zeta_from_PO = 0.5  # Default value if PO is invalid

    # Plotting
    plt.contourf(R, I, (R**2 + I**2) < omega_n_settle**2, levels=[0, 1], colors='blue', alpha=0.3)
    plt.axhline(0, color='black', lw=0.5, ls='--')
    plt.axvline(0, color='black', lw=0.5, ls='--')

    # Calculate and plot poles based on zeta
    for omega_n in np.linspace(0.1, 3, 30):
        pole_real = -zeta * omega_n
        pole_imag = omega_n * np.sqrt(1 - zeta**2)
        plt.plot(pole_real, pole_imag, 'ro')

        # Calculate theta, real part and print them
        theta = np.arctan2(pole_imag, pole_real) * 180 / np.pi  # Convert radians to degrees
        real_part = pole_real

        # Annotate the angle and real part on the plot
        plt.annotate(f'θ={theta:.1f}°\nRe={real_part:.2f}\nω_n={omega_n:.2f}',
                     xy=(pole_real, pole_imag), xytext=(10, 10),
                     textcoords='offset points', fontsize=8, color='black',
                     arrowprops=dict(arrowstyle='->', color='black'))

    # Draw a circle based on a chosen radius
    circle_radius = 1.0  # You can modify this radius
    circle = plt.Circle((0, 0), circle_radius, color='green', fill=False, linestyle='--', linewidth=1.5)
    plt.gca().add_artist(circle)

    plt.title('Allowed Pole Locations in the s-plane')
    plt.xlim(-3, 1)
    plt.ylim(-3, 3)
    plt.xlabel('Real Axis')
    plt.ylabel('Imaginary Axis')
    plt.grid()
    plt.gca().set_aspect('equal', adjustable='box')  # Ensure aspect ratio is equal
    plt.show()

# Create interactive widgets for user input
K_slider = widgets.FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='K:')
zeta_slider = widgets.FloatSlider(value=0.5, min=0.01, max=1.0, step=0.01, description='Damping Ratio (zeta):')
T_s_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Settling Time (T_s):')
PO_slider = widgets.FloatSlider(value=5.0, min=0.1, max=100.0, step=0.1, description='Peak Overshoot (%):')
T_r_slider = widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description='Rise Time (T_r):')

interactive_plot = interactive(plot_poles, K=K_slider, zeta=zeta_slider, T_s=T_s_slider, PO=PO_slider, T_r=T_r_slider)
display(interactive_plot)


interactive(children=(FloatSlider(value=1.0, description='K:', max=10.0, min=0.1), FloatSlider(value=0.5, desc…